# <img src="https://minmod.isi.edu/resource/staticResources/img/minmod-logo.png" alt="minmod kg" width="25"/> MinMod Knowledge Graph

### live at [https://minmod.isi.edu/](https://minmod.isi.edu/) wrapping a [SPARQL Endpoint](https://minmod.isi.edu/sparql).

MinMod is the mineral data **Knowledge Graph (KG)**, integrating heterogeneous data sources including: grade-tonnage data extracted from **mine reports**, **scholarly articles**, **mine site databases**, and **structured tables** to provide a rich, queryable graph of mineral site information, and **links** to additional knowledge bases such as [GeoKB](https://geokb.wikibase.cloud/).

### data in this knowledge graph adhere to this [schema](https://github.com/DARPA-CRITICALMAAS/schemas/blob/main/ta2/README.md).

## Why KGs?

<img src="demo_imgs/minmod_kg.png" alt="minmod kg" width="250"/>

- KGs
  - graphs are natural way to **encode data**
  - KGs use **semantic concepts & relationships** to create a **Semantic Network**
  - involves **spatial & temporal** information
- RDF
  - framework within the **Semantic** Web stack
  - an extension of WWW, enabling the Web of Data (aka **"Linked Data"**)
  - Linked Open Data & **FAIR** data principles

## Constructing the KG

- Extracted data --> `JSON-LD` readers / `TTL` triples reader
- Predefined data
  - Open set of commodity entities (based on MRDS/GeoKB)
  - Finite set of deposit type entities
  - Ontology following schema to enforce class & property constraints

<img src="demo_imgs/minmod_pipeline.png" alt="minmod kg" width="600"/>

## Interacting with the KG
MinMod KG `SPARQL` Sandbox

In [1]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import Counter
import warnings

warnings.filterwarnings("ignore")
tqdm.pandas()

In [9]:
def run_sparql_query(query, endpoint='https://minmod.isi.edu/sparql', values=False):
    # add prefixes
    final_query = '''
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <https://minmod.isi.edu/resource/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX gkbi: <https://geokb.wikibase.cloud/entity/>
    PREFIX gkbp: <https://geokb.wikibase.cloud/wiki/Property:>
    PREFIX gkbt: <https://geokb.wikibase.cloud/prop/direct/>
    \n''' + query
    # send query
    response = requests.post(
        url=endpoint,
        data={'query': final_query},
        headers={
            "Content-Type": "application/x-www-form-urlencoded",
            "Accept": "application/sparql-results+json"  # Requesting JSON format
        },
        verify=False  # Set to False to bypass SSL verification as per the '-k' in curl
    )
    #print(response.text)
    try:
        qres = response.json()
        if "results" in qres and "bindings" in qres["results"]:
            df = pd.json_normalize(qres['results']['bindings'])
            if values:
                filtered_columns = df.filter(like='.value').columns
                df = df[filtered_columns]
            return df
    except:
        return None

In [4]:
def run_minmod_query(query, values=False):
    return run_sparql_query(query, endpoint='https://minmod.isi.edu/sparql', values=values)

def run_geokb_query(query, values=False):
    return run_sparql_query(query, endpoint='https://geokb.wikibase.cloud/query/sparql', values=values)

--------------------------------------------------------

### 0. Count total number of triples in KG

In [18]:
query = ''' SELECT (COUNT(?s) as ?count)
            WHERE {
                ?s ?p ?o .
            } '''
run_minmod_query(query)

,count.type,count.datatype,count.value
0,literal,http://www.w3.org/2001/XMLSchema#integer,1345


### 1. Deposit Types

In [23]:
query = ''' SELECT ?ci ?cn
            WHERE {
                ?ci a :DepositType .
                ?ci :name ?cn .
            } '''
run_minmod_query(query)

,ci.type,ci.value,cn.type,cn.value
0,uri,https://minmod.isi.edu/resource/Q4,literal,Sedex Type Deposits
1,uri,https://minmod.isi.edu/resource/Q24,literal,MVT zinc-lead


### 2. Mineral Inventories

#### 2.1. all **inferred** ore values, from all inventories, their grades, & cutoff grades

In [32]:
query = ''' SELECT ?o_inv ?comm_name ?ore ?grade ?cutoff_grade
            WHERE {
                ?s :mineral_inventory ?o_inv .
                ?o_inv :category :Inferred .
                ?o_inv :commodity [ :name ?comm_name ] .
                ?o_inv :ore [ :ore_value ?ore ] .
                ?o_inv :grade [ :grade_value ?grade ] .
                ?o_inv :cutoff_grade [ :grade_value ?cutoff_grade ] .
            } '''
run_minmod_query(query, values=True)

,o_inv.value,comm_name.value,ore.value,grade.value,cutoff_grade.value
0,https://minmod.isi.edu/resource/Q104012,Zinc,1970400,14.36,8
1,https://minmod.isi.edu/resource/Q104024,Zinc,2393400,12.98,5
2,https://minmod.isi.edu/resource/Q104016,Zinc,2100600,13.94,7
3,https://minmod.isi.edu/resource/Q10404,Zinc,1499200,16.02,10
4,https://minmod.isi.edu/resource/Q104032,Zinc,2824300,11.6,3
5,https://minmod.isi.edu/resource/Q10408,Zinc,1772600,15.01,9
6,https://minmod.isi.edu/resource/Q114,Zinc,2276600,13.37,6
7,https://minmod.isi.edu/resource/Q104020,Zinc,2276000,13.37,6
8,https://minmod.isi.edu/resource/Q104028,Zinc,2887100,11.88,4


#### 2.2. all mineral inventories with **inferred** grade >= 13

In [30]:
query = ''' SELECT ?o_inv ?comm_name ?ore ?grade
            WHERE {
                ?s :mineral_inventory ?o_inv .
                ?o_inv :category :Inferred .
                ?o_inv :commodity [ :name ?comm_name ] .
                ?o_inv :ore [ :ore_value ?ore ] .
                ?o_inv :grade [ :grade_value ?grade ] .
                FILTER (?grade >= 14)
            } '''
run_minmod_query(query, values=True)

,o_inv.value,comm_name.value,ore.value,grade.value
0,https://minmod.isi.edu/resource/Q104012,Zinc,1970400,14.36
1,https://minmod.isi.edu/resource/Q10404,Zinc,1499200,16.02
2,https://minmod.isi.edu/resource/Q10408,Zinc,1772600,15.01


### 3. commodities

#### 3.1. all commodities and their `GeoKB` URIs

In [13]:
query = ''' SELECT ?ci ?cn ?gi
            WHERE {
                ?ci a :Commodity .
                ?ci :name ?cn .
                ?ci owl:sameAs ?gi .
            } '''
example_df = run_minmod_query(query)
example_df

,ci.type,ci.value,cn.type,cn.xml:lang,cn.value,gi.type,gi.value
0,uri,https://minmod.isi.edu/resource/Q563,literal,en,"Iron, Pyrite",uri,https://geokb.wikibase.cloud/entity/Q448
1,uri,https://minmod.isi.edu/resource/Q515,literal,en,Boron-Borates,uri,https://geokb.wikibase.cloud/entity/Q418
2,uri,https://minmod.isi.edu/resource/Q574,literal,en,Mica,uri,https://geokb.wikibase.cloud/entity/Q476
3,uri,https://minmod.isi.edu/resource/Q541,literal,en,Dolomite,uri,https://geokb.wikibase.cloud/entity/Q444
4,uri,https://minmod.isi.edu/resource/Q565,literal,en,Lead,uri,https://geokb.wikibase.cloud/entity/Q342
...,...,...,...,...,...,...,...
84,uri,https://minmod.isi.edu/resource/Q544,literal,en,Gemstone,uri,https://geokb.wikibase.cloud/entity/Q452
85,uri,https://minmod.isi.edu/resource/Q568,literal,en,"Limestone, Ultra Pure",uri,https://geokb.wikibase.cloud/entity/Q471
86,uri,https://minmod.isi.edu/resource/Q535,literal,en,"Coal, Lignite",uri,https://geokb.wikibase.cloud/entity/Q438
87,uri,https://minmod.isi.edu/resource/Q516,literal,en,Cadmium,uri,https://geokb.wikibase.cloud/entity/Q306


#### 3.2. get commodity symboles from `GeoKB`

In [12]:
def get_symbol_via_sparql(geokb_uri):
    query = '''
    SELECT ?symb
    WHERE {
        <%s> gkbt:P17 ?symb .
    }''' % (geokb_uri)
    result_record = run_geokb_query(query)
    if len(result_record) > 0:
        return result_record.iloc[0]['symb.value']
    return ""

In [14]:
pd.set_option('display.max_rows', 500)

example_df['geoKB Symbol'] = example_df['gi.value'].progress_apply(get_symbol_via_sparql)
example_df

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 89/89 [00:53<00:00,  1.67it/s]


,ci.type,ci.value,cn.type,cn.xml:lang,cn.value,gi.type,gi.value,geoKB Symbol
0,uri,https://minmod.isi.edu/resource/Q563,literal,en,"Iron, Pyrite",uri,https://geokb.wikibase.cloud/entity/Q448,
1,uri,https://minmod.isi.edu/resource/Q515,literal,en,Boron-Borates,uri,https://geokb.wikibase.cloud/entity/Q418,
2,uri,https://minmod.isi.edu/resource/Q574,literal,en,Mica,uri,https://geokb.wikibase.cloud/entity/Q476,
3,uri,https://minmod.isi.edu/resource/Q541,literal,en,Dolomite,uri,https://geokb.wikibase.cloud/entity/Q444,
4,uri,https://minmod.isi.edu/resource/Q565,literal,en,Lead,uri,https://geokb.wikibase.cloud/entity/Q342,Pb
5,uri,https://minmod.isi.edu/resource/Q555,literal,en,Gypsum-Anhydrite,uri,https://geokb.wikibase.cloud/entity/Q463,
6,uri,https://minmod.isi.edu/resource/Q518,literal,en,Cement Rock,uri,https://geokb.wikibase.cloud/entity/Q420,
7,uri,https://minmod.isi.edu/resource/Q548,literal,en,"Gemstone, Sapphire",uri,https://geokb.wikibase.cloud/entity/Q457,
8,uri,https://minmod.isi.edu/resource/Q587,literal,en,Tin,uri,https://geokb.wikibase.cloud/entity/Q392,Sn
9,uri,https://minmod.isi.edu/resource/Q539,literal,en,"Copper, Oxide",uri,https://geokb.wikibase.cloud/entity/Q441,


------